In [1]:
# ! pip install pip install PyPortfolioOpt

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import gmean
from pypfopt.hierarchical_portfolio import HRPOpt
from datetime import date


C:\Users\Nilakshi Mondal\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.8' currently installed).
  warnings.warn(msg, UserWarning)


## Data Handling

In [172]:
data = pd.read_csv('data.csv').dropna()
data.set_index('timeclose',inplace=True)
data.reset_index(inplace=True)

#Format Dates by removing timestamps

data.rename(columns={'timeclose':'Date'},inplace=True)
data['Date'] = pd.to_datetime(data['Date']).dt.date
data['Date'] = data['Date'].astype('datetime64[ns]')



In [173]:
#Averaging over timestamps in a day

data = data.groupby('Date').mean()
data.reset_index(inplace=True)

In [174]:
# Add columns for years and months

data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month

years= sorted(data.Year.unique())
months= sorted(data.Month.unique())


In [175]:
data

,Date,ETH_returns,WBTC_returns,XRP_returns,MATIC_returns,ADA_returns,BNB_returns,KAVA_returns,BTC_returns,SHIB_returns,USDC_returns,HNT_returns,ATOM_returns,PAXG_returns,DOGE_returns,SOL_returns,Year,Month
0,2020-08-01,0.005048,0.000706,0.005404,0.001385,0.001280,0.001794,-0.002092,0.001981,0.029649,-2.375611e-04,-0.003954,0.002489,-0.000211,0.006463,-0.003087,2020,8
1,2020-08-02,-0.037984,-0.027432,-0.054318,-0.021378,-0.039333,-0.038410,-0.035086,-0.025852,-0.026884,7.660027e-03,0.019560,-0.045074,-0.000620,-0.034235,-0.029090,2020,8
2,2020-08-03,0.001749,0.000970,0.003264,0.000954,0.001188,0.002183,0.001698,0.000728,-0.046163,1.128134e-04,0.003317,-0.000371,-0.000009,0.000662,0.001577,2020,8
3,2020-08-04,0.000433,0.000085,-0.001232,0.000389,0.001740,0.000456,0.005844,-0.000142,0.006316,8.120125e-08,0.002781,0.001679,0.001065,0.001827,0.001728,2020,8
4,2020-08-05,0.001251,0.001509,0.000417,0.001486,-0.000083,0.001965,0.003074,0.001975,0.031664,3.346677e-06,0.003696,0.001563,0.000282,-0.000244,-0.002566,2020,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
737,2022-10-06,-0.000026,-0.000448,0.000313,-0.000900,-0.000245,-0.001043,-0.000927,-0.000421,-0.000933,1.301513e-06,-0.004146,0.000015,-0.000128,-0.000824,-0.000898,2022,10
738,2022-10-07,-0.000586,-0.000795,0.002102,-0.000049,-0.000424,-0.000498,0.000302,-0.000852,-0.000325,2.206854e-06,-0.002135,0.000202,-0.000327,-0.000675,-0.000415,2022,10
739,2022-10-08,-0.000532,-0.000325,-0.000084,-0.000921,-0.000253,-0.000994,0.000310,-0.000277,-0.000332,5.238289e-06,-0.001967,0.000230,-0.000054,-0.000488,-0.000535,2022,10
740,2022-10-09,0.000226,0.000096,0.001366,0.000601,0.000133,0.000267,0.000081,0.000065,0.000135,-6.417510e-06,-0.000043,0.000263,0.000050,0.000324,0.000431,2022,10


In [185]:
df = []
for i in years:
    for j in months:
        d = data[(data['Month'] == j) & (data['Year']==i)]
        if d.empty:
            continue
        else:
            df.append(d)

In [4]:
# Number of days in dataset

# from datetime import date

# d0 = date(2020,8,1)
# d1 = date(2022,10,11)
# delta = d1 - d0
# n_days = delta.days
# n_days

801

In [191]:

# tickers = []

# for i in range (0,len(data.columns)):
#     tckr = data.columns[i].rstrip("_returns")
#     tickers.append(tckr)
    

## Hierarchial Risk Parity (for entire data)

In [205]:
#HRP Model

new_data = data.drop(columns=['Month','Year'])
new_data.set_index('Date', inplace=True)
HRP = HRPOpt(new_data,new_data.cov)

In [206]:
#Weights for Minimum Var Portfolio  using Hierarchial Risk Parity Algorithm
w_vector = HRP.optimize(linkage_method='single')
weights= w_vector.values()


In [208]:
#Model Performance for total number of days in dataset 

HRP.portfolio_performance(weights, risk_free_rate= 0.036)

Expected annual return: 0.1%
Annual volatility: 0.3%
Sharpe Ratio: -10.35


0.003355069272074405

## Monthly HRP

In [211]:
monthly_hrp_vol = []

for i in range(0,len(df)):
    dataset= df[i]
    HRP = HRPOpt(dataset,dataset.cov)
    w_vector = HRP.optimize(linkage_method='single')
    weights= w_vector.values()
    performance = HRP.portfolio_performance(weights, risk_free_rate= 0.036, frequency = 21)
    vol = performance[1]
    monthly_hrp_vol.append(vol)



ValueError: The condensed distance matrix must contain only finite values.

In [10]:
HRP = HRPOpt(data,np.cov)
w_vector = HRP.optimize(linkage_method='single')
weights= w_vector.values()
HRP.portfolio_performance(weights, risk_free_rate= 0.036, frequency = 21)

Expected annual return: 0.1%
Annual volatility: 0.4%
Sharpe Ratio: -8.08


(0.0011257856409899744, 0.004316677052961862, -8.078949138685575)

##  Markowitz

In [11]:
# Returns

returns = data.mean()
# data.apply(gmean)
#geomtric means cannot be applied or negative numbers/zeros. In that case need log returns instead of pct_change for prices

In [12]:
# Weights



In [13]:
# Markovitz Model




In [14]:
for i  in range(0, 1001, 1):
    print (i / 1000)

0.0
0.001
0.002
0.003
0.004
0.005
0.006
0.007
0.008
0.009
0.01
0.011
0.012
0.013
0.014
0.015
0.016
0.017
0.018
0.019
0.02
0.021
0.022
0.023
0.024
0.025
0.026
0.027
0.028
0.029
0.03
0.031
0.032
0.033
0.034
0.035
0.036
0.037
0.038
0.039
0.04
0.041
0.042
0.043
0.044
0.045
0.046
0.047
0.048
0.049
0.05
0.051
0.052
0.053
0.054
0.055
0.056
0.057
0.058
0.059
0.06
0.061
0.062
0.063
0.064
0.065
0.066
0.067
0.068
0.069
0.07
0.071
0.072
0.073
0.074
0.075
0.076
0.077
0.078
0.079
0.08
0.081
0.082
0.083
0.084
0.085
0.086
0.087
0.088
0.089
0.09
0.091
0.092
0.093
0.094
0.095
0.096
0.097
0.098
0.099
0.1
0.101
0.102
0.103
0.104
0.105
0.106
0.107
0.108
0.109
0.11
0.111
0.112
0.113
0.114
0.115
0.116
0.117
0.118
0.119
0.12
0.121
0.122
0.123
0.124
0.125
0.126
0.127
0.128
0.129
0.13
0.131
0.132
0.133
0.134
0.135
0.136
0.137
0.138
0.139
0.14
0.141
0.142
0.143
0.144
0.145
0.146
0.147
0.148
0.149
0.15
0.151
0.152
0.153
0.154
0.155
0.156
0.157
0.158
0.159
0.16
0.161
0.162
0.163
0.164
0.165
0.166
0.167
0.168
0.169

Notes

Start with good data, feed HRPOPt only a month's worth of data and find out vol for next month, try for a year, predict for next year. find out what kind of vol HRP is giving, (daily, yearly or monthly) . end output: graph predictions for predictions, historical and predictions 